In [1]:
import pandas as pd
import os
import timeit
import numpy as np
import random
import swifter
import time
import seaborn as sns

c:\Users\Computadores Gamer\AppData\Local\Programs\Python\Python311\Lib\site-packages\dask\dataframe\_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 13.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(
c:\Users\Computadores Gamer\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
diretorio = r'C:\Users\Computadores Gamer\OneDrive\Área de Trabalho\apply-if\dados'
diretorio = diretorio.replace('\\','//')

os.chdir(diretorio)
os.getcwd()

'C:\\Users\\Computadores Gamer\\OneDrive\\Área de Trabalho\\apply-if\\dados'

In [12]:
#### MORADOR
# largura do txt
larguras = [2,4,1,9,2,1,2,2,1,2,2,4,3,1,1,
            1,1,1,2,1,2,1,1,1,1,1,1,1,1,1,
            1,1,1,1,1,2,1,1,2,1,1,2,1,1,1,
            2,1,2,14,14,10,1,20,20,20,20]

# nome das colunas
colunas = ["UF", "ESTRATO_POF", "TIPO_SITUACAO_REG",
            "COD_UPA", "NUM_DOM", "NUM_UC", "COD_INFORMANTE",
            "V0306", "V0401", "V04021", "V04022", "V04023",
            "V0403", "V0404", "V0405", "V0406", "V0407",
            "V0408", "V0409", "V0410", "V0411", "V0412",
            "V0413", "V0414", "V0415", "V0416",
            "V041711", "V041712", "V041721", "V041722",
            "V041731", "V041732", "V041741", "V041742",
            "V0418", "V0419", "V0420", "V0421", "V0422",
            "V0423", "V0424", "V0425", "V0426", "V0427",
            "V0428", "V0429", "V0430", "ANOS_ESTUDO",
            "PESO", "PESO_FINAL", "RENDA_TOTAL",
            "NIVEL_INSTRUCAO", "RENDA_DISP_PC","RENDA_MONET_PC",
            "RENDA_NAO_MONET_PC","DEDUCAO_PC" ]

# leitura dos dados
MORADOR = pd.read_fwf(
    os.path.join(diretorio, "MORADOR.txt"),
    widths=larguras,
    na_values=[" "],
    names=colunas,
    decimal="."
)

# Condições

In [4]:
condicao1 = ((MORADOR['RENDA_DISP_PC'] + MORADOR['PESO_FINAL']>1000) &\
    (MORADOR['RENDA_MONET_PC'] > 300) &\
    (MORADOR['RENDA_NAO_MONET_PC'] < 200) &\
    (MORADOR['RENDA_TOTAL'] < 1000))

condicao2 = ((MORADOR['RENDA_TOTAL'] * 5 > 500) &\
        (MORADOR['RENDA_MONET_PC'] > MORADOR['RENDA_TOTAL'] / 2))

# Função

In [5]:
def funcao(z):
    if (z['RENDA_DISP_PC'] + z['PESO_FINAL'])>1000 and\
        (z['RENDA_MONET_PC']> 300) and\
        (z['RENDA_NAO_MONET_PC']< 200) and\
        z['RENDA_TOTAL']<1000:
        return 1
    
    elif z['RENDA_TOTAL']* 5 > 500 and\
        (z['RENDA_MONET_PC'] > z['RENDA_TOTAL']/2):
        return 1
    
    else:    
        return 0

In [13]:
tempo = []

# For

In [11]:
inicio = time.time()
for i in range(len(MORADOR)):
    if( MORADOR.loc[i,'RENDA_DISP_PC'] + MORADOR.loc[i,'PESO_FINAL']>1000) and\
        (MORADOR.loc[i,'RENDA_MONET_PC']>300) and\
        (MORADOR.loc[i,'RENDA_NAO_MONET_PC']<200) and\
        (MORADOR.loc[i,'RENDA_TOTAL']<1000):
        MORADOR.loc[i,'var_teste'] = 1
    
    elif MORADOR.loc[i,'RENDA_TOTAL']*5 >500 and\
        MORADOR.loc[i,'RENDA_MONET_PC'] > MORADOR.loc[i,'RENDA_TOTAL']/2:
        MORADOR.loc[i,'var_teste'] = 1
    
    else:
        MORADOR.loc[i,'var_teste'] = 0      
final = time.time()

delta_for = final-inicio
tempo.append(delta_for)
MORADOR['var_teste'].value_counts()

var_teste
0.0    170714
1.0      7717
Name: count, dtype: int64

# Apply

In [ ]:
inicio = time.time()
MORADOR['var_teste'] = MORADOR.apply(funcao, axis=1)
final = time.time()

delta_apply = final-inicio
tempo.append(delta_apply)
MORADOR['var_teste'].value_counts()

# Vetorização pandas

In [ ]:
inicio = time.time()
MORADOR.loc[condicao1, 'var_teste']=1
# MORADOR.loc[condicao2, 'var_teste']=0
final = time.time()

delta_vetor_pandas = final-inicio
tempo.append(delta_vetor_pandas)
MORADOR['var_teste'].value_counts()


# Funcao para vetorização numpy

In [ ]:
def function1(a, b, c, d, e):
    if a + b > 1000 and c > 300 and d < 200 and e < 1000:
        return 1
    elif e * 5 > 500 and c > e / 2:
        return 1
    else:
        return 0
   
    
function_vetorizada = np.vectorize(pyfunc = function1)


# Vetorização numpy

In [ ]:
# MORADOR['var_teste'] = ''
inicio = time.time()
function_vetorizada = (MORADOR['RENDA_DISP_PC'], MORADOR['PESO_FINAL'],MORADOR['RENDA_MONET_PC'],MORADOR['RENDA_NAO_MONET_PC'],MORADOR['RENDA_TOTAL'])
final = time.time()

delta_vetor_numpy = final-inicio
tempo.append(delta_vetor_numpy)


# np.select

In [14]:
inicio = time.time()
MORADOR['var_teste'] = np.select([condicao1, condicao2], [1, 0])
final = time.time()

delta_select = final-inicio
tempo.append(delta_select)
MORADOR['var_teste'].value_counts()

var_teste
0    177978
1       453
Name: count, dtype: int64

# np.where

In [8]:
inicio = time.time()
MORADOR['var_teste'] = np.where(condicao1, True , np.where(condicao2, False, 0))
final = time.time()

delta_where = final-inicio
tempo.append(delta_where)
MORADOR['var_teste'].value_counts()

var_teste
0    177978
1       453
Name: count, dtype: int64

# Tabela resumo dos tempos

In [ ]:
tabela_tempo = pd.DataFrame({'tempo':tempo}, index =['for', 'apply', 'vetor pandas','vetor numpy', 'np.select','np.where'])
tabela_tempo

# Gráfico

In [ ]:
sns.lineplot(data = tabela_tempo, x=tabela_tempo.index, y='tempo')